# Regularization Mode

## Theory

## Example

In [1]:
from saeno import FiniteBodyForces
    
# initialize the object
M = FiniteBodyForces()

Set the material model:

In [2]:
from saeno.materials import SemiAffineFiberMaterial

# provide a material model
material = SemiAffineFiberMaterial(1645, 0.0008, 0.0075, 0.033)
M.setMaterialModel(material)

Load the mesh from files.

In [3]:
import numpy as np

# define the coordinates of the nodes of the mesh
# the array has to have the shape N_v x 3
R = np.array([[0., 0., 0.],  # 0
              [0., 1., 0.],  # 1
              [1., 1., 0.],  # 2
              [1., 0., 0.],  # 3
              [0., 0., 1.],  # 4
              [1., 0., 1.],  # 5
              [1., 1., 1.],  # 6
              [0., 1., 1.]]) # 7

# define the tetrahedra of the mesh
# the array has to have the shape N_t x 4
# every entry is an index referencing a verces in R (indices start with 0)
T = np.array([[0, 1, 7, 2],
              [0, 2, 5, 3],
              [0, 4, 5, 7],
              [2, 5, 6, 7],
              [0, 7, 5, 2]])

And hand the data over to the FiniteBodyForces object.

In [4]:
# provide the node data
M.setNodes(R)
# and the tetrahedron data
M.setTetrahedra(T)

Now we have to specify which displacements to fit.

In [5]:
# the displacements of the nodes which shall be fitted
# during the solving
U = np.array([[0   , 0, 0],  # 0
              [0   , 0, 0],  # 1
              [0.01, 0, 0],  # 2
              [0.01, 0, 0],  # 3
              [0   , 0, 0],  # 4
              [0.01, 0, 0],  # 5
              [0.01, 0, 0],  # 6
              [0   , 0, 0]]) # 7

# hand the displacements over to the class instance
M.setTargetDisplacements(U)

Now we can start the regularisation process.

In [6]:
# relax the mesh and move the "varible" nodes
M.regularize(stepper=0.1, alpha=0.001)

going to update glo f and K
updating forces and stiffness matrix finished 3.57s
|u-uf|^2 = 0.0004 		perbead= 0.005
|w*f|^2  = 4.405430147324622e-28 		|u|^2 = 0.0
L = |u-uf|^2 + lambda*|w*f|^2 =  0.0004
check before relax !
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.06s
Round 1  |du|= 0.0005215094932618363
|u-uf|^2 = 0.0003531282756934563 		perbead= 0.00489013892067284
|w*f|^2  = 0.0010996365051104385 		|u|^2 = 2.1757772124977385e-06
L = |u-uf|^2 + lambda*|w*f|^2 =  0.00035422791219856677
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.08s
Round 2  |du|= 0.00046825541087788157
|u-uf|^2 = 0.00031512129237042184 		perbead= 0.004797556845255541
|w*f|^2  = 0.003663715074098005 		|u|^2 = 7.83671901007029e-06
L = |u-uf|^2 + lambda*|w*f|^2 =  0.00031878500744451986
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.07s
Round 3  |du|= 0.00042057632420637044
|u-uf|^2 = 0.0002842840803025308 		perbead= 0.004719449322628919
|w*

[(0.0004, 0.0004, 4.405430147324622e-28),
 (0.00035422791219856677, 0.0003531282756934563, 0.0010996365051104385),
 (0.00031878500744451986, 0.00031512129237042184, 0.003663715074098005),
 (0.00029119894700023826, 0.0002842840803025308, 0.00691486669770747),
 (0.00026963303810503093, 0.00025924982387480315, 0.010383214230227785),
 (0.00025271039846563555, 0.00023891524076373753, 0.013795157701898032),
 (0.00023938999248175263, 0.00022238901576524228, 0.01700097671651036),
 (0.00022887845900022215, 0.00020895054170212636, 0.01992791729809578),
 (0.00022056680874549224, 0.00019801684188080912, 0.02254996686468313),
 (0.0002139845462169495, 0.000189116001370309, 0.0248685448466405),
 (0.00020876608088555153, 0.00018186580141914775, 0.026900279466403776),
 (0.00020462591546130377, 0.00017595651410179833, 0.02866940135950545),
 (0.0002013401180098697, 0.0001711370415411673, 0.030203076468702413),
 (0.0001987323979304949, 0.000167203723850868, 0.03152867407962691),
 (0.00019666352896323898, 

Now we can optinally calculate some properties of the cell, e.g. its contractility and polarisation.

In [7]:
results = M.computeForceMoments(rmax=100e-6)

d:\repositories\saeno_py\saeno\FiniteBodyForces.py:772: RuntimeWarning: invalid value encountered in true_divide
  contractility = np.sum(np.einsum("ki,ki->k", RR, f) / np.linalg.norm(RR, axis=1))
d:\repositories\saeno_py\saeno\FiniteBodyForces.py:778: RuntimeWarning: invalid value encountered in true_divide
  eR = RR / np.linalg.norm(RR, axis=1)[:, None]
d:\repositories\saeno_py\saeno\FiniteBodyForces.py:795: RuntimeWarning: invalid value encountered in true_divide
  vmid = vmid / np.linalg.norm(vmid)


In [8]:
# store the forces of the nodes
M.storeF("F.dat")
# store the positions and the displacements
M.storeRAndU("R.dat", "U.dat")
# store the center of each tetrahedron and a combined list with energies and volumina of the tetrahedrons
M.storeEandV("RR.dat", "EV.dat")

F.dat stored.
R.dat stored.
U.dat stored.
RR.dat stored.
EV.dat stored.


In [9]:
M.U
#M._updateGloFAndK()

array([[ 4.01026265e-03,  7.91968666e-05,  7.90874559e-05],
       [ 2.88113112e-03, -5.89581479e-04,  5.89400217e-04],
       [ 4.32201918e-03, -7.91968666e-05,  7.90874559e-05],
       [ 5.45115071e-03,  5.89581479e-04,  5.89400217e-04],
       [ 2.88113112e-03,  5.89581479e-04, -5.89400217e-04],
       [ 4.32201918e-03,  7.91968666e-05, -7.90874559e-05],
       [ 5.45115071e-03, -5.89581479e-04, -5.89400217e-04],
       [ 4.01026265e-03, -7.91968666e-05, -7.90874559e-05]])

In [10]:
M.viewMesh(10, 1)